# Random search

In [1]:
import sympy as sp
import numpy as np

LIMITS_RANGE = -3, 3

COMPONENTS = 3
DATA_SIZE = 10

In [2]:
syms = sp.symbols(f'x1:{COMPONENTS+1}')

In [3]:
import operator
import functools

x1 = syms[0]
x2 = syms[1]
x3 = syms[2]

eq = sum(syms)
# eq = eq.subs(x1, 1/x1**2).subs(x2, x2**2).subs(x3, x3**3)
# eq = functools.reduce(operator.mul, syms, 1)

eq

x1 + x2 + x3

In [4]:
f = sp.lambdify(syms, eq, modules='numpy')

# Code

In [15]:
import typing
import math
import collections
import dataclasses
import abc
import itertools

T = typing.TypeVar("T", int, float)
Vector = typing.Tuple[T, T]
VectorComponent = typing.List[T] | typing.Tuple[T, ...]
Vertex = typing.List[typing.Tuple[T, T]] | typing.Tuple[typing.Tuple[T, T]]

def product_combination(iterable):
    first = iterable[0]

    for r in itertools.product(first, *iterable[1:]):
        yield r

def check_axis_intersect(that: Vector, z: T):
    x, y = that
    return y >= z >= x


def is_collision(that: Vertex, other: VectorComponent) -> bool:
    if len(that) != len(other):
        raise ValueError("Invalid arguments, different length of values")

    return all([check_axis_intersect(t, z) for t, z in zip(that, other)])


@dataclasses.dataclass
class VectorNode:
    data: VectorComponent
    value: T

    def __gt__(self, other):
        return self.value > other.value

    def __str__(self):
        return f"{self.data}, {self.value}"

    def __iter__(self):
        return iter([*self.data, self.value])


def create_axis(_array: Vertex):
    all_data = []

    for arr in _array:
        x, y = arr
        d = math.fabs(y - x) / 2

        all_data.append(((x, x + d), (x + d, y)))

    return [*product_combination(all_data)]


@dataclasses.dataclass
class Node:
    vertex: Vertex
    nodes = []

    def append(self, node: VectorComponent):
        self.nodes.append(node)

    def is_collide(self, node: VectorComponent):
        return is_collision(self.vertex, node)


class NodeContainerInterface(abc.ABC):
    pass


class NodeContainer:
    def __init__(self, data: Vertex, limit_divisions=1):
        self.__node = Node(data)
        self.__children = None

        if limit_divisions > 0:
            self.__children = [
                NodeContainer(vertices, limit_divisions - 1) for vertices in create_axis(data)
            ]

    @property
    def children(self) -> typing.List:
        if self.__children:
            return [*self.__children]

        return []

    @property
    def node(self) -> Node:
        return self.__node

    @property
    def is_parent(self):
        return len(self.children) != 0

    def insert(self, verx: VectorComponent) -> bool:
        return self._insert_loop(verx)

    def _insert_loop(self, verx: VectorComponent) -> bool:
        children = collections.deque(self.children)

        while True:
            child = children.pop()
            node = child.node

            if node.is_collide(verx):
                if child.is_parent:
                    children = child.children

                    continue
                else:
                    node.append(verx)
                    print("Insert in:", node.vertex, verx)

                    return True

            if len(children) <= 0:
                return False

    def _insert_recursive(self, verx: VectorComponent) -> bool:
        for child in self.children:
            node = child.node

            if node.is_collide(verx):
                if child.is_parent:
                    return child.insert(verx)
                else:
                    node.append(verx)
                    print("Insert in:", node.vertex, verx)

                    return True

        return False



class TreeNode:
    def __init__(self):
        pass

node = NodeContainer(((0, 1), (0, 1), (0, 1), (0, 1)), 2)
node.insert((.5, .2, .1, .1))

Insert in: ((0.5, 0.75), (0, 0.25), (0, 0.25), (0, 0.25)) (0.5, 0.2, 0.1, 0.1)


True

In [7]:
import pandas as pd

def tree_search_algorithm(n_it=500):
    all_data = []

    for it in range(n_it):
        data = [np.random.uniform(-3, 3) for _ in range(COMPONENTS)]
        value = f(*data)

        all_data.append(
            VectorNode(data, value)
        )

    return [list(vec) for vec in all_data], min(all_data)

all_data, min_found = tree_search_algorithm()

In [8]:
pd.DataFrame(all_data)

,0,1,2,3
0,1.816534,1.750941,-0.031048,3.536427
1,1.518266,0.582514,-0.109688,1.991092
2,-2.646383,0.486215,2.447695,0.287528
3,-2.213744,2.279481,1.525135,1.590872
4,-1.541561,-2.164690,2.705212,-1.001040
...,...,...,...,...
495,-2.584497,-2.135489,0.070864,-4.649121
496,0.771934,1.134022,2.468837,4.374793
497,1.239973,1.750466,0.989425,3.979865
498,-0.103125,0.252162,-1.406339,-1.257302


In [9]:
min_found

VectorNode(data=[-2.694655117192016, -2.9646614710760977, -2.6922710921026973], value=-8.35158768037081)